In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import filter_data_by_year_month, create_features_from_past, create_X_y



In [2]:
X = pd.read_csv('/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT')


In [3]:
YEARS = [2024]

Data = filter_data_by_year_month(X, YEARS)

Data1 = create_features_from_past(Data,['close', 'open', 'high', 'low', 'volume'], 4)

In [4]:
Data1.head()

,timestamp,open,high,low,close,volume,lag_close1,lag_open1,lag_high1,lag_low1,...,lag_close3,lag_open3,lag_high3,lag_low3,lag_volume3,lag_close4,lag_open4,lag_high4,lag_low4,lag_volume4
222911,2024-01-01 01:00:00,42475.23,42475.23,42431.65,42466.33,188.76099,42475.23,42441.32,42490.74,42422.45,...,42419.73,42488.00,42554.57,42412.02,392.24889,42488.00,42283.58,42488.09,42261.02,431.71082
222912,2024-01-01 01:15:00,42466.32,42513.17,42466.32,42493.16,152.48691,42466.33,42475.23,42475.23,42431.65,...,42441.32,42419.73,42447.82,42354.19,319.90644,42419.73,42488.00,42554.57,42412.02,392.24889
222913,2024-01-01 01:30:00,42493.16,42775.00,42493.16,42697.50,642.57307,42493.16,42466.32,42513.17,42466.32,...,42475.23,42441.32,42490.74,42422.45,127.81493,42441.32,42419.73,42447.82,42354.19,319.90644
222914,2024-01-01 01:45:00,42697.50,42720.00,42613.56,42613.56,212.55759,42697.50,42493.16,42775.00,42493.16,...,42466.33,42475.23,42475.23,42431.65,188.76099,42475.23,42441.32,42490.74,42422.45,127.81493
222915,2024-01-01 02:00:00,42613.57,42638.41,42541.10,42556.63,217.29821,42613.56,42697.50,42720.00,42613.56,...,42493.16,42466.32,42513.17,42466.32,152.48691,42466.33,42475.23,42475.23,42431.65,188.76099


In [ ]:
num_nan_rows = Data1.isna().any(axis=1).sum()
print(num_nan_rows)

In [5]:
X,y = create_X_y(Data1)

In [16]:
X.head()

,lag_close1,lag_open1,lag_high1,lag_low1,lag_volume1,lag_close2,lag_open2,lag_high2,lag_low2,lag_volume2,lag_close3,lag_open3,lag_high3,lag_low3,lag_volume3,lag_close4,lag_open4,lag_high4,lag_low4,lag_volume4
222911,42475.23,42441.32,42490.74,42422.45,127.81493,42441.32,42419.73,42447.82,42354.19,319.90644,42419.73,42488.00,42554.57,42412.02,392.24889,42488.00,42283.58,42488.09,42261.02,431.71082
222912,42466.33,42475.23,42475.23,42431.65,188.76099,42475.23,42441.32,42490.74,42422.45,127.81493,42441.32,42419.73,42447.82,42354.19,319.90644,42419.73,42488.00,42554.57,42412.02,392.24889
222913,42493.16,42466.32,42513.17,42466.32,152.48691,42466.33,42475.23,42475.23,42431.65,188.76099,42475.23,42441.32,42490.74,42422.45,127.81493,42441.32,42419.73,42447.82,42354.19,319.90644
222914,42697.50,42493.16,42775.00,42493.16,642.57307,42493.16,42466.32,42513.17,42466.32,152.48691,42466.33,42475.23,42475.23,42431.65,188.76099,42475.23,42441.32,42490.74,42422.45,127.81493
222915,42613.56,42697.50,42720.00,42613.56,212.55759,42697.50,42493.16,42775.00,42493.16,642.57307,42493.16,42466.32,42513.17,42466.32,152.48691,42466.33,42475.23,42475.23,42431.65,188.76099


In [17]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35132 entries, 222911 to 258042
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lag_close1   35132 non-null  float64
 1   lag_open1    35132 non-null  float64
 2   lag_high1    35132 non-null  float64
 3   lag_low1     35132 non-null  float64
 4   lag_volume1  35132 non-null  float64
 5   lag_close2   35132 non-null  float64
 6   lag_open2    35132 non-null  float64
 7   lag_high2    35132 non-null  float64
 8   lag_low2     35132 non-null  float64
 9   lag_volume2  35132 non-null  float64
 10  lag_close3   35132 non-null  float64
 11  lag_open3    35132 non-null  float64
 12  lag_high3    35132 non-null  float64
 13  lag_low3     35132 non-null  float64
 14  lag_volume3  35132 non-null  float64
 15  lag_close4   35132 non-null  float64
 16  lag_open4    35132 non-null  float64
 17  lag_high4    35132 non-null  float64
 18  lag_low4     35132 non-null  float64
 19  lag

In [ ]:
y.head()

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [7]:
# Calculate sizes for train and validation sets
dataset_size = len(X)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [13]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [
        Dense(16, activation='relu', input_shape=(20,), name="Input_Layer"),  # Input layer with 21 features
        Dropout(0.3),
        Dense(8, activation='relu', name="Hidden_Layer1"),  # Another hidden layer
        Dense(1, activation='linear', name="Output_Layer")  # Output layer for regression
    ],
    name="Price_Prediction_Model"
)

In [ ]:
model.summary()

In [ ]:
model = tf.keras.models.load_model("../models/model_1.h5") #IF you need to dowload a model 

In [15]:
model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    metrics=['mae']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=6,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/Final_training/third_model_5.keras',             # File to save the best model
    monitor='val_accuracy',       # Metric to monitor for improvement
    mode='max',                   # Mode 'max' for accuracy (since higher is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)

history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=200,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/200
879/879 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 212157872.0000 - mae: 10059.0820 - val_loss: 218609952.0000 - val_mae: 10239.2998 - learning_rate: 0.1000
Epoch 2/200
879/879 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 216117936.0000 - mae: 10174.5820 - val_loss: 219153952.0000 - val_mae: 10359.4717 - learning_rate: 0.1000
Epoch 3/200
879/879 ━━━━━━━━━━━━━━━━━━━━ 1s 855us/step - loss: 214656864.0000 - mae: 10157.0635 - val_loss: 211154672.0000 - val_mae: 10076.2070 - learning_rate: 0.1000
Epoch 4/200
879/879 ━━━━━━━━━━━━━━━━━━━━ 1s 878us/step - loss: 216507776.0000 - mae: 10233.3291 - val_loss: 218344272.0000 - val_mae: 10190.1279 - learning_rate: 0.1000
Epoch 5/200
879/879 ━━━━━━━━━━━━━━━━━━━━ 1s 858us/step - loss: 219021792.0000 - mae: 10288.2939 - val_loss: 223274240.0000 - val_mae: 10366.2354 - learning_rate: 0.1000
Epoch 6/200
879/879 ━━━━━━━━━━━━━━━━━━━━ 1s 888us/step - loss: 215774720.0000 - mae: 10170.3594 - val_loss: 214110192.0000 - val_mae: 10161.5576 - learning_rat

In [ ]:
model.save("../models/model_1.h5")  # Saves the model as an HDF5 file
